In [54]:
import pandas as pd
import numpy as np
import openpyxl
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
pre_train = pd.read_csv('train.csv')
pre_test = pd.read_csv('test.csv')

In [3]:
pre_dataset = pd.concat([pre_train, pre_test], axis=0, sort=False)

In [4]:
pre_dataset

,building_id,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,txn_floor,land_area,building_area,town,lat,lon,village_income_median,town_population,town_area,town_population_density,doc_rate,master_rate,bachelor_rate,jobschool_rate,highschool_rate,junior_rate,elementary_rate,born_rate,death_rate,marriage_rate,divorce_rate,village,N_50,N_500,N_1000,N_5000,N_10000,I_10,I_50,I_index_50,I_100,I_250,I_500,I_index_500,I_1000,I_index_1000,I_5000,I_index_5000,I_10000,I_index_10000,I_MIN,II_10,II_50,II_index_50,II_100,II_250,II_500,II_index_500,II_1000,II_index_1000,II_5000,II_index_5000,II_10000,II_index_10000,II_MIN,III_10,III_50,III_index_50,III_100,III_250,III_500,III_index_500,III_1000,III_index_1000,III_5000,III_index_5000,III_10000,III_index_10000,III_MIN,IV_10,IV_50,IV_index_50,IV_100,IV_250,IV_500,IV_index_500,IV_1000,IV_index_1000,IV_5000,IV_index_5000,IV_10000,IV_index_10000,IV_MIN,V_10,V_50,V_index_50,V_100,V_250,V_500,V_index_500,V_1000,V_index_1000,V_5000,V_index_5000,V_10000,V_index_10000,V_MIN,VI_10,VI_50,VI_index_50,VI_100,VI_250,VI_500,VI_index_500,VI_1000,VI_index_1000,VI_5000,VI_index_5000,VI_10000,VI_index_10000,VI_MIN,VII_10,VII_50,VII_index_50,VII_100,VII_250,VII_500,VII_index_500,VII_1000,VII_index_1000,VII_5000,VII_index_5000,VII_10000,VII_index_10000,VII_MIN,VIII_10,VIII_50,VIII_index_50,VIII_100,VIII_250,VIII_500,VIII_index_500,VIII_1000,VIII_index_1000,VIII_5000,VIII_index_5000,VIII_10000,VIII_index_10000,VIII_MIN,IX_10,IX_50,IX_index_50,IX_100,IX_250,IX_500,IX_index_500,IX_1000,IX_index_1000,IX_5000,IX_index_5000,IX_10000,IX_index_10000,IX_MIN,X_10,X_50,X_index_50,X_100,X_250,X_500,X_index_500,X_1000,X_index_1000,X_5000,X_index_5000,X_10000,X_index_10000,X_MIN,XI_10,XI_50,XI_index_50,XI_100,XI_250,XI_500,XI_index_500,XI_1000,XI_index_1000,XI_5000,XI_index_5000,XI_10000,XI_index_10000,XI_MIN,XII_10,XII_50,XII_index_50,XII_100,XII_250,XII_500,XII_index_500,XII_1000,XII_index_1000,XII_5000,XII_index_5000,XII_10000,XII_index_10000,XII_MIN,XIII_10,XIII_50,XIII_index_50,XIII_100,XIII_250,XIII_500,XIII_index_500,XIII_1000,XIII_index_1000,XIII_5000,XIII_index_5000,XIII_10000,XIII_index_10000,XIII_MIN,XIV_10,XIV_50,XIV_index_50,XIV_100,XIV_250,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN,total_price
0,e3mMIMR3JJqCaXz1,8,21,18674,4,3,2,6271,2,NaN,NaN,3.0,18.144460,3.418175,334,-39.14,117.08,572.0,24219.0,1.4161,17103.0,0.003948,0.047944,0.208219,0.102360,0.327584,0.166931,0.143015,7.37,7.72,5.74,2.39,3132,2,14,14,14,14,0,0,0,1,5,20,1,59,1,399,1,779,1,84.745794,0,0,0,0,4,9,1,37,1,805,1,1992,1,102.554396,0,1,1,3,17,37,1,91,1,1861,1,3728,1,42.635964,0,0,0,0,0,7,1,18,1,224,1,299,1,287.735804,0,0,0,0,2,23,1,56,1,1077,1,2001,1,205.295496,0,0,0,0,0,5,1,17,1,97,1,201,1,270.838262,0,0,0,0,17,93,1,277,1,4028,1,6989,1,116.075571,0,0,0,1,15,85,1,160,1,3432,1,6607,1,69.868801,0,0,0,1,5,40,1,79,1,1110,1,1944,1,68.178741,0,0,0,2,3,16,1,43,1,807,1,1620,1,64.774668,0,0,0,0,6,18,1,56,1,853,1,1886,1,132.498164,0,0,0,5,25,91,1,203,1,4342,1,9448,1,63.478618,0,0,0,0,1,2,1,8,1,147,1,295,1,112.582703,0,1,1,3,21,58,1,157,1,2483,1,6011,1,34.469803,6.476038e+05
1,LgwzgklNvy4QCtq5,8,7,18800,5,1,2,7885,2,NaN,NaN,5.0,11.387227,4.041309,180,-37.66,119.28,550.0,551480.0,23.1373,23835.0,0.003518,0.048779,0.216042,0.106792,0.310975,0.195193,0.118700,7.94,5.93,6.11,2.50,921,5,12,14,14,14,0,0,0,1,2,5,1,13,1,487,1,1334,1,85.529684,0,0,0,3,8,18,1,41,1,1691,1,4304,1,71.994648,1,1,1,1,4,11,1,29,1,2512,1,7380,1,7.157464,0,0,0,0,0,0,0,7,1,94,1,492,1,805.037288,0,0,0,1,7,11,1,26,1,1799,1,5331,1,65.829031,0,0,0,0,0,0,0,5,1,77,1,354,1,719.071571,1,1,1,3,18,31,1,94,1,5050,1,18608,1,6.864786,0,0,0,2,9,16,1,76,1,5204,1,15064,1,65.829031,0,0,0,0,5,7,1,17,1,1349,1,4399,1,102.299486,0,0,0,0,0,4,1,25,1,1084,1,3004,1,272.553558,0,0,0,0,6,11,1,24,1,949,1,3053,1,125.670173,0,3,1,14,45,81,1,285,1,9846,1,25456,1,23.717447,0,1,1,2,4,5,1,1

In [5]:
pre_dataset['parking_price'] = pre_dataset['parking_price'].fillna(int(0))
pre_dataset['parking_area'] = pre_dataset['parking_area'].fillna(int(0))
pre_dataset['txn_floor'] = pre_dataset['txn_floor'].fillna(int(0))

# pre_train = pre_train.drop(['lat','lon'], axis=1)

In [6]:
dummy_fields = ['building_material', 'city', 'total_floor', 'building_type', 
                'building_use', 'parking_way', 'txn_floor', 'town', 'village']

for each in dummy_fields:
    dummies = pd.get_dummies(pre_dataset.loc[:, each], prefix=each) 
    dum = pd.concat([pre_dataset, dummies], axis = 1)

In [25]:
dataset = dum.drop(dummy_fields, axis = 1 )
dataset.shape

(70000, 3173)

In [31]:
train_set = dataset.iloc[:60000, :]
test_set = dataset.iloc[60000:, :]
train_set.shape, test_set.shape

((60000, 3173), (10000, 3173))

In [32]:
train_set_dropna = train_set.dropna(axis=1)
test_set_dropna = test_set.dropna(axis=1)
train_set_dropna.shape, test_set_dropna.shape

((60000, 3172), (10000, 3171))

In [40]:
train_set_ID = train_set_dropna['building_id']
test_set_ID = test_set_dropna['building_id']

In [45]:
X = train_set_dropna.drop(['building_id', 'total_price'], axis=1)
y = train_set_dropna['total_price']

X_test = test_set_dropna.drop(['building_id'], axis=1)
X.shape, y.shape, X_test.shape

((60000, 3170), (60000,), (10000, 3170))

In [46]:
regressor_trees = RandomForestRegressor(n_estimators = 1400, min_samples_split = 5,
                                        min_samples_leaf = 1, max_features = 'sqrt',
                                        max_depth = 30, bootstrap = True)
regressor_trees.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=1400, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [50]:
trees_pred = regressor_trees.predict(X_test)
type(trees_pred)

numpy.ndarray

In [52]:
trees_pred = trees_pred.tolist()
trees_pred.insert(0, 'total_price')

In [57]:
# wb = openpyxl.load_workbook("submit_test.xlsx")
# ws = wb['submit_test']

# for i in range(0, len(trees_pred)):
#     ws.cell(row= i+1, column = 2).value = trees_pred[i]
# wb.save("submit_test_v2.xlsx")

In [58]:
test_set_ID.head()

0    X5gsdTWGS3W7JJQB
1    BTshNOJyKHnT2YIT
2    dhdymr0lV8N5kZOT
3    VEwyGGMcD56w5BOc
4    wmUeMoJZfsqaSX9b
Name: building_id, dtype: object

In [59]:
submit_test = pd.read_csv('submit_test.csv')
submit_test.head()

,building_id,total_price
0,X5gsdTWGS3W7JJQB,0
1,BTshNOJyKHnT2YIT,0
2,dhdymr0lV8N5kZOT,0
3,VEwyGGMcD56w5BOc,0
4,wmUeMoJZfsqaSX9b,0


In [ ]:
# regressor_trees = BaggingRegressor(DecisionTreeRegressor(splitter='random', max_leaf_nodes=16),
#                   n_estimators = 500, max_samples = 1.0, bootstrap = True, random_state = 0)

# regressor_trees.fit(X_train, y_train)
# trees_pred = regressor_trees.predict(X_test)

In [ ]:
y_test_np = y_test.values
y_test_np[0]

In [ ]:
(trees_pred[0] - y_test_np[0]) / y_test_np[0]

In [ ]:
for price in trees_pred:
    if price < 0:
        print(price)

In [ ]:
hit_rate = []
mape = []
for i in range(0, len(trees_pred)):
    mape.append(abs((trees_pred[i] - y_test_np[i]) / y_test_np[i]))
    if abs((trees_pred[i] - y_test_np[i]) / y_test_np[i]) <= 0.1:
        hit_rate.append(int(1))
    else:
        hit_rate.append(int(0))
hit_rate_all = sum(np.array(hit_rate)) / len(trees_pred)
mape_all = sum(np.array(mape)) / len(trees_pred)

In [ ]:
if mape_all >= 1:
    score = hit_rate_all*10000 + (1-1)
else:
    score = hit_rate_all*10000 + (1-mape_all)

In [ ]:
score